## Jul AdventKalender D24

https://adventofcode.com/2022/day/24

In [1]:
import heapq

#### Day 24.1 

Given a map of a valley (# is the walls) and blizzards with four directions (< left, > right, ^ up, v down). 

* In one minute, each blizzard moves **one position** in the **direction** it is pointing.
* As a blizzard **reaches the wall** of the valley, a **new** blizzard forms on the **opposite side** of the valley moving in the **same direction**.
* Expedition begins in the only non-wall position in the top row and needs to reach the only non-wall position in the bottom row. 
* On each minute, you can move **up**, **down**, **left**, or **right**, or you can **wait** in place. 
* You and the blizzards act **simultaneously**, and you cannot share a position with a blizzard. But blizzard can share position with another blizzard.

For example:

    #S######
    #>>.<^<#
    #.<..<<#
    #>v.><>#
    #<^v^^>#
    ######E#

What is the fewest number of minutes required to avoid the blizzards and reach the goal?

In [2]:
def readMap(file_name):
    data_map = []
    f = open(file_name, "r")
    while True:
        line = f.readline()
        if not line:
            break
        data_map.append(line.strip()[1:-1])
    f.close()
    del data_map[0]
    del data_map[-1]
    return data_map

In [3]:
def _noBlizzardComing(pos, step_id):
    global blizzard_map, num_rows, num_cols
    x, y = pos
    step_id += 1
    directions = [
        blizzard_map[(x-step_id)%num_rows][y] == 'v',    # up comes blizzard
        blizzard_map[(x+step_id)%num_rows][y] == '^',    # down 
        blizzard_map[x][(y-step_id)%num_cols] == '>',    # left 
        blizzard_map[x][(y+step_id)%num_cols] == '<',    # right
    ]
    return sum(directions) == 0

def _isInBounds(pos):
    global num_rows, num_cols
    x, y = pos
    return 0 <= x < num_rows and 0 <= y < num_cols

def _getNeighbors(pos_c, step_id):
    global pos_s, pos_e
    x, y = pos_c
    neighbors = [
        (x, y),   # stay
        (x-1, y), # up   
        (x+1, y), # down 
        (x, y-1), # left
        (x, y+1), # right
    ]
    neighbors_valid = []
    for n in neighbors:
        if (_isInBounds(n) and _noBlizzardComing(n, step_id)) or n == pos_s or n == pos_e:
            neighbors_valid.append(n)
    return neighbors_valid

def _heuristic(a, b):
    (x1, y1) = a
    (x2, y2) = b
    return abs(x1 - x2) + abs(y1 - y2)
                
def findShortestPath(pos_s, pos_e, step_id_s = 0): # a*
    visited = set()
    queue = []
    heapq.heappush(queue, (0, pos_s, step_id_s)) # priority, position, step_id
    while queue:
        _, node, step_id = heapq.heappop(queue)
        if node == pos_e:
            return step_id
        if (node, step_id) in visited:
            continue
        visited.add((node, step_id))
        neighbors = _getNeighbors(node, step_id)
        #print('current pos:', node, 'step_id:', step_id, 'valid neighbors:', neighbors)
        for neighbor in neighbors:
            priority = step_id + _heuristic(pos_e, neighbor)
            heapq.heappush(queue, (priority, neighbor, step_id+1))
    return None

In [4]:
blizzard_map = readMap("data/input24.txt")
num_rows = len(blizzard_map)
num_cols = len(blizzard_map[0])
pos_s, pos_e = (-1,0), (num_rows,num_cols-1)

n_steps_1 = findShortestPath(pos_s, pos_e)
print('Fewest steps required to reach the goal:', n_steps_1)

Fewest steps required to reach the goal: 314


#### Day 24.2

What is the fewest number of minutes required to reach the goal, go back to the start, then reach the goal again?

In [5]:
n_steps_back = findShortestPath(pos_e, pos_s, n_steps_1)
n_steps_2 = findShortestPath(pos_s, pos_e, n_steps_back)
print('Fewest steps required to reach the goal, go back to start, and reach the goal again:', n_steps_2)

Fewest steps required to reach the goal, go back to start, and reach the goal again: 896
